In [1]:
## Importing the libraries 
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model

In [3]:
#Loading the dataset
insurance=pd.read_csv("insurance.csv")

In [4]:
### Printing the first five rows
insurance.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [5]:
### Printing the last five rows
insurance.tail()

,age,sex,bmi,children,smoker,region,expenses
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95
1337,61,female,29.1,0,yes,northwest,29141.36


In [6]:
### Printing the summary of the dataframe
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [7]:
### Finding the correlation between the columns
insurance.corr()

,age,bmi,children,expenses
age,1.000000,0.109341,0.042469,0.299008
bmi,0.109341,1.000000,0.012645,0.198576
children,0.042469,0.012645,1.000000,0.067998
expenses,0.299008,0.198576,0.067998,1.000000


In [8]:
### Printing all the column names in the dataset
insurance.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses'], dtype='object')

In [9]:
## Checking if there are any NULL values
insurance.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [10]:
### Label Encoding
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
insurance['sex'] = labelencoder_y.fit_transform(insurance['sex'])

In [11]:
def map_smoking(column):
    mapped=[]
    
    for row in column:
        
        if row=="yes":
            mapped.append(1)
        else:
            mapped.append(0)
        
        
    return mapped
insurance["smoker_norm"]=map_smoking(insurance["smoker"])

In [12]:
nonnum_cols=[col for col in insurance.select_dtypes(include=["object"])]

In [13]:
def map_obese(column):
    mapped=[]
    for row in column:
        if row>30:
            mapped.append(1)
        else:
            mapped.append(0)
    return mapped
insurance["obese"]=map_obese(insurance["bmi"])

In [14]:
### Printing the first five rows of the updated dataset
insurance.head()

,age,sex,bmi,children,smoker,region,expenses,smoker_norm,obese
0,19,0,27.9,0,yes,southwest,16884.92,1,0
1,18,1,33.8,1,no,southeast,1725.55,0,1
2,28,1,33.0,3,no,southeast,4449.46,0,1
3,33,1,22.7,0,no,northwest,21984.47,0,0
4,32,1,28.9,0,no,northwest,3866.86,0,0


In [15]:
### Plotting the graphs using scatter plot
colnum=len(insurance.columns)-3
fig,ax=plt.subplots(colnum,1,figsize=(3,25))
ax[0].set_ylabel("charges")
p_vals={}
for ind,col in enumerate([i for i in insurance.columns 
                          if i not in ["smoker","region","charges","sex_norm"]])
    
    ax[ind].scatter(insurance[col],insurance.charges,s=5)
    ax[ind].set_xlabel(col)
    ax[ind].set_ylabel("charges")    
plt.show()

SyntaxError: invalid syntax (<ipython-input-15-7a46855e79ea>, line 7)

In [16]:
### Dropping the unwanted columns
insurance.drop('bmi', inplace=True, axis=1)
insurance.drop('smoker', inplace=True, axis=1 )
insurance.drop('region', inplace=True, axis=1 )


In [17]:
###Printing the updated dataset
insurance

,age,sex,children,expenses,smoker_norm,obese
0,19,0,0,16884.92,1,0
1,18,1,1,1725.55,0,1
2,28,1,3,4449.46,0,1
3,33,1,0,21984.47,0,0
4,32,1,0,3866.86,0,0
...,...,...,...,...,...,...
1333,50,1,3,10600.55,0,1
1334,18,0,0,2205.98,0,1
1335,18,0,0,1629.83,0,1
1336,21,0,0,2007.95,0,0


In [18]:
###Choosing the Dependent and Independent variables
# Independent Variables
x=insurance.iloc[:,[0,1,2,4,5]]
x

,age,sex,children,smoker_norm,obese
0,19,0,0,1,0
1,18,1,1,0,1
2,28,1,3,0,1
3,33,1,0,0,0
4,32,1,0,0,0
...,...,...,...,...,...
1333,50,1,3,0,1
1334,18,0,0,0,1
1335,18,0,0,0,1
1336,21,0,0,0,0


In [19]:
# Dependent Variable(target column)
y=insurance.iloc[:,3]
y

0       16884.92
1        1725.55
2        4449.46
3       21984.47
4        3866.86
          ...   
1333    10600.55
1334     2205.98
1335     1629.83
1336     2007.95
1337    29141.36
Name: expenses, Length: 1338, dtype: float64

In [20]:
## TRAIN TEST SPLIT

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


In [21]:
## FEATURE SCALING

from sklearn.preprocessing import MinMaxScaler

sc=MinMaxScaler()


In [22]:
x_train

,age,sex,children,smoker_norm,obese
621,37,1,4,1,1
194,18,1,0,0,1
240,23,0,2,1,1
1168,32,1,2,0,1
1192,58,0,1,0,1
...,...,...,...,...,...
763,27,1,0,0,0
835,42,1,2,0,1
1216,40,1,0,0,0
559,19,1,0,0,1


In [23]:
x_train=sc.fit_transform(x_train)

In [24]:
x_train

array([[0.41304348, 1.        , 0.8       , 1.        , 1.        ],
       [0.        , 1.        , 0.        , 0.        , 1.        ],
       [0.10869565, 0.        , 0.4       , 1.        , 1.        ],
       ...,
       [0.47826087, 1.        , 0.        , 0.        , 0.        ],
       [0.02173913, 1.        , 0.        , 0.        , 1.        ],
       [0.32608696, 0.        , 0.2       , 0.        , 0.        ]])

In [25]:
x_test=sc.transform(x_test)

x_test

array([[0.73913043, 1.        , 0.2       , 0.        , 1.        ],
       [0.63043478, 0.        , 0.2       , 0.        , 0.        ],
       [0.65217391, 1.        , 0.4       , 1.        , 1.        ],
       ...,
       [0.84782609, 1.        , 0.        , 0.        , 1.        ],
       [0.26086957, 0.        , 0.6       , 1.        , 1.        ],
       [0.60869565, 1.        , 0.6       , 0.        , 0.        ]])

In [26]:
y_test

578      9724.53
610      8547.69
569     45702.02
1034    12950.07
198      9644.25
          ...   
1084    15019.76
726      6664.69
1132    20709.02
725     40932.43
963      9500.57
Name: expenses, Length: 268, dtype: float64

In [27]:
## MODEL BUILDING
from sklearn.linear_model import LinearRegression
lr=linear_model.LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [28]:
y_pred=lr.predict(x_test)
y_pred

array([13699.88808451,  8139.28029997, 36701.24089254, 15568.42476135,
        8726.05321143,  4398.31755147,   266.41235998, 14290.5503908 ,
       10957.96460084,  9193.75647161,  5683.97071167,  9346.97666002,
        7113.86928767,  4111.70365609, 26191.97399944,  9785.61476086,
       11063.4891314 ,  7034.71650458,  6301.00476543, 28646.08760479,
       33221.7162749 , 15133.67605534, 13230.98884138, 31288.76791863,
        5570.66739145,  8572.83302302,  2829.93989072, 11470.67010698,
        3598.99814994, 12344.05691382,  7222.08723011, 39082.48307553,
       15824.77751442, 14030.3082429 , 25392.65459792,  3342.64539687,
       12600.4096669 , 31135.54773022, 34620.15772565,  5027.700743  ,
        4880.76191532,  3116.5537861 , 29268.20703634, 39264.76842329,
       28103.12095634,  6161.32969774,  9785.61476086,  6887.7776769 ,
        3781.2834977 , 10702.80783071,  5796.75900222,  1838.67941555,
       32014.01991484, 37878.6761103 , 14769.10535983,  7108.78390989,
      

In [29]:
## Testing for Accuracy
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.7995703673276304

In [30]:
## Predicting and the result
lr.predict([[37,1,4,1,1]])

array([473218.52984806])

In [33]:
#Saving our model into a file
import pickle
pickle.dump(lr,open('HIC.pkl','wb'))